# Fit emimTF2N data

EoS: PR

Parameter T dependence: Quadratic

N (total fitting parameters): 6

## Import Functions

In [1]:
import idaes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

import sys
sys.path.append('../../../')

from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.opt import TerminationCondition, SolverStatus

from idaes.core import FlowsheetBlock
import idaes.logger as idaeslog
# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
# Import unit models from the model library
from idaes.generic_models.unit_models import Flash
# Import degrees of freedom tool
from idaes.core.util.model_statistics import degrees_of_freedom

# parmest (binary_param2)
from bip_fitting_functions import cuadratic

import scipy.stats as stats

## Load Data

In [2]:
data_full = pd.read_csv('../r125_emimtf2n_full.csv')
data_subset = pd.read_csv('../r125_emimtf2n_subset.csv')
data_extended = pd.read_csv('../r125-emimtf2n-extended.csv')

## Load Configuration + Check DOF + Load Parameters

In [3]:
lhs_results1 = pd.read_csv('../Data/Fits/LHS_Fits/PR_quadTdep_LHS.csv',index_col=0)
lhs_results1 = lhs_results1[['sc_param1','sc_param2','sc_param3','sc_param4','sc_param5','sc_param6','SSR']]
lhs_results2 = pd.read_csv('Data/Init_Final/LHS_Output/PR_quadTdep_LHS_final.csv',index_col=0)
lhs_results2 = lhs_results2[['sc_param1','sc_param2','sc_param3','sc_param4','sc_param5','sc_param6','SSR']]

In [4]:
lhs_results = pd.concat([lhs_results1,lhs_results2],axis=0).reset_index(drop=True)

In [5]:
lhs_results.dropna(axis=0)

,sc_param1,sc_param2,sc_param3,sc_param4,sc_param5,sc_param6,SSR
11,-0.932453,-0.459271,-0.911759,-0.513797,0.692297,0.936531,11.317269
227,0.140683,-0.506367,-0.463518,0.552634,0.371398,-0.011327,11.317269
317,-0.287150,-1.549336,-1.177858,-0.571437,0.740480,1.468091,5053.244800
777,-1.416031,-1.550567,-0.251962,-0.119530,1.175178,1.663826,1758.237809
1041,-1.773214,-1.330504,-0.322407,-0.503838,0.674388,1.898514,11.317269
1148,-0.009158,-1.450230,-1.394972,1.209682,-1.639140,0.197456,11.317269
1255,-1.450291,-1.289904,1.465798,1.675292,0.427188,-0.674721,234.245233
1510,-1.644923,-1.195560,0.502120,0.238762,-0.607318,0.786893,3237.162013
1751,-0.496796,0.589789,0.292200,-1.622518,-0.059634,1.001379,11.317267
1806,-1.955602,-1.210803,1.545365,-1.850051,-0.423649,1.731783,2832.394749


In [6]:
lhs_best = lhs_results[lhs_results.SSR == lhs_results.SSR.min()]
lhs_best

,sc_param1,sc_param2,sc_param3,sc_param4,sc_param5,sc_param6,SSR
1751,-0.496796,0.589789,0.2922,-1.622518,-0.059634,1.001379,11.317267


In [7]:
sc_param1 = lhs_best.at[1751, 'sc_param1']
sc_param2 = lhs_best.at[1751, 'sc_param2']
sc_param3 = lhs_best.at[1751, 'sc_param3']
sc_param4 = lhs_best.at[1751, 'sc_param4']
sc_param5 = lhs_best.at[1751, 'sc_param5']
sc_param6 = lhs_best.at[1751, 'sc_param6']

In [8]:
sc_param1

-0.4967961034287567

In [9]:
from hfc125_emimtf2n_PR import configuration 

In [10]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
m.fs.properties = GenericParameterBlock(default=configuration)
m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})
DOF_initial = degrees_of_freedom(m)
print('The initial DOF is {0}'.format(DOF_initial))

The initial DOF is 7


## Fit Parameters

In [11]:
parameters, obj_value, a = cuadratic(data_subset, configuration, 'R125', 'emimTf2N', "x_R125", "x_emimTf2N", 
        init_temp =  283.1, init_press =   399300 , init_x_c1 =    0.448, init_x_c2 = 0.552,
        init_kappa_2_1A = sc_param1, init_kappa_1_2A = sc_param2, 
        init_kappa_2_1B = sc_param3, init_kappa_1_2B = sc_param4,
        init_kappa_2_1C = sc_param5, init_kappa_1_2C = sc_param6,
        eps = 0.1, scaling_fac = 1e-7 , filename='Data/Fits/Ipopt_Output/PR_quadTdep_final.txt')

2022-10-14 12:17:05 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:17:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:17:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:17:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:17:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:17:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:17:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:17:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-10-14 12:17:08 [INFO] idaes

/Users/bridgettebefort/opt/anaconda3/envs/idaes-pse-dev/lib/python3.9/site-packages/pyomo/contrib/interior_point/interface.py:418: RuntimeWarning: divide by zero encountered in true_divide
  data = (duals_primals_lb/(primals - self._nlp.primals_lb()) +


In [12]:
print('Objective (SSE):',obj_value)

Objective (SSE): 11.317266936398706


In [13]:
print('Parameters:',parameters)

Parameters: fs.properties.PR_kappa_A[emimTf2N,R125]    0.793442
fs.properties.PR_kappa_A[R125,emimTf2N]    0.110739
fs.properties.PR_kappa_B[emimTf2N,R125]   -1.594267
fs.properties.PR_kappa_B[R125,emimTf2N]   -0.272435
fs.properties.PR_kappa_C[emimTf2N,R125]    0.736423
fs.properties.PR_kappa_C[R125,emimTf2N]    0.141693
dtype: float64


In [14]:
params2save = np.zeros(8)
params2save[0] = parameters[1]
params2save[1] = parameters[0]
params2save[2] = parameters[3]
params2save[3] = parameters[2]
params2save[4] = parameters[5]
params2save[5] = parameters[4]

In [15]:
np.savetxt('Data/Fits/PR_params_quadTdep.csv',params2save,delimiter=',')

In [16]:
cov = np.array(a)
print('Covariance Matrix:')
print(cov)

Covariance Matrix:
[[ 6.93087435e+00  2.27506993e-01 -1.38082898e+01 -4.51562958e-01
   6.86097747e+00  2.23333377e-01]
 [ 2.27506993e-01  2.87333663e-02 -4.29205756e-01 -5.39705381e-02
   2.00963758e-01  2.51929733e-02]
 [-1.38082898e+01 -4.29205756e-01  2.75629417e+01  8.56107016e-01
  -1.37229520e+01 -4.25529943e-01]
 [-4.51562959e-01 -5.39705381e-02  8.56107016e-01  1.01582151e-01
  -4.03071701e-01 -4.75188540e-02]
 [ 6.86097747e+00  2.00963758e-01 -1.37229520e+01 -4.03071701e-01
   6.84688468e+00  2.01474052e-01]
 [ 2.23333377e-01  2.51929733e-02 -4.25529943e-01 -4.75188540e-02
   2.01474052e-01  2.22790423e-02]]


In [17]:
np.savetxt('Data/Covariance/PR_Cov_quadTdep.csv',cov,delimiter=',')

## Apply Parameters + Plot

In [ ]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [ ]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.properties = GenericParameterBlock(default=configuration)

m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m.fs.F101.inlet.flow_mol.fix(1)
m.fs.F101.inlet.temperature.fix(323.15)
m.fs.F101.inlet.pressure.fix(399800)
m.fs.F101.inlet.mole_frac_comp[0,'R125'].fix(0.5)

m.fs.properties.PR_kappa_A['R125','emimTf2N'] = parameters['fs.properties.PR_kappa_A[R125,emimTf2N]']
m.fs.properties.PR_kappa_A['emimTf2N','R125'] = parameters['fs.properties.PR_kappa_A[emimTf2N,R125]']
m.fs.properties.PR_kappa_B['R125','emimTf2N'] = parameters['fs.properties.PR_kappa_B[R125,emimTf2N]']
m.fs.properties.PR_kappa_B['emimTf2N','R125'] = parameters['fs.properties.PR_kappa_B[emimTf2N,R125]']
m.fs.properties.PR_kappa_C['R125','emimTf2N'] = parameters['fs.properties.PR_kappa_C[R125,emimTf2N]']
m.fs.properties.PR_kappa_C['emimTf2N','R125'] = parameters['fs.properties.PR_kappa_C[emimTf2N,R125]']

# Initialize the flash unit
m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)

# Fix the state variables on the state block
# m.fs.F101.inlet.pressure.unfix()
m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].unfix()

# data_extended = data_extended[::-1]
R125_x = np.zeros((len(data_extended)))
P = np.zeros((len(data_extended)))
T = np.zeros((len(data_extended)))

m.fs.liq = Param(mutable=True,default=0.040)
m.fs.liquid = Constraint(expr=m.fs.F101.liq_outlet.mole_frac_comp[0, "emimTf2N"] == m.fs.liq)

for i in range(len(data_extended)):
    print('i:',i)
    print('Try x_IL:',data_extended["x_emimTf2N"].iloc[i])
    m.fs.liq = data_extended["x_emimTf2N"].iloc[i]
    m.fs.F101.inlet.temperature.fix(float(data_extended["temperature"].iloc[i]))
    m.fs.F101.inlet.pressure.fix(float(data_extended["pressure"].iloc[i]))
    m.fs.F101.inlet.mole_frac_comp[0,'R125'].fix(float(data_extended["x_R125"].iloc[i])+0.1)
    m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].fix(float(1-(data_extended["x_R125"].iloc[i]+0.1)))
    m.fs.F101.vap_outlet.temperature.fix(float(data_extended["temperature"].iloc[i]))
    
    try:
        m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
        
    except:
        print('Initialization Error')
        
    try:

        status = solver.solve(m, tee = False)
        
        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):
            print('Feasible/Optimal Solution')
            R125_x[i] = value(m.fs.F101.liq_outlet.mole_frac_comp[0,'R125']) 
            P[i] = value(m.fs.F101.vap_outlet.pressure[0])
            T[i] = data_extended["temperature"].iloc[i]
        else:
            print('Infeasible Solution')
            R125_x[i] = np.nan
            P[i] = np.nan
            T[i] = data_extended["temperature"].iloc[i]
            
    except ValueError:
        R125_x[i] = np.nan
        P[i] = np.nan
        T[i] = data_extended["temperature"].iloc[i]
        print('Solver Error')

In [ ]:
fitted_data = pd.DataFrame(data=[R125_x,P/1e6,T])
fitted_data = fitted_data.T
fitted_data.columns = ['R125_x','P, MPa','T, K']
fitted_data.to_csv('Data/Fits/PR_FittedData_quadTdep.csv')

In [ ]:
plt.plot(fitted_data.where(fitted_data['T, K']==283.1)['R125_x'],fitted_data.where(fitted_data['T, K']==283.1)['P, MPa'],"b-",label='T=283 K')
plt.plot(fitted_data.where(fitted_data['T, K']==298.1)['R125_x'],fitted_data.where(fitted_data['T, K']==298.1)['P, MPa'],"r-",label='T=298 K')
plt.plot(fitted_data.where(fitted_data['T, K']==323.1)['R125_x'],fitted_data.where(fitted_data['T, K']==323.1)['P, MPa'],"c-",label='T=323 K')
plt.plot(fitted_data.where(fitted_data['T, K']==348.1)['R125_x'],fitted_data.where(fitted_data['T, K']==348.1)['P, MPa'],"m-",label='T=348 K')
plt.plot(data_full["x_R125"],data_full["pressure"]/1e6,"*",color='orange',label="data")  
plt.title('Isotherm R125-[emim][TF2N]')
plt.ylabel('Pressure (MPa)')
plt.xlabel('x R125')
plt.grid(True)
plt.ylim(0,1.110000)
plt.legend()
plt.show()

### Print Eigvals, Eigvectors

An identifiability analysis can be performed with these as well as a FIM because the covariance matrix is the inverse of the FIM. That means that if there are any eigvals > 0, their eigvecs indicate directions of un-identifiability.

In [ ]:
eigvals, eigvecs = np.linalg.eig(cov)

In [ ]:
eigvals

In [ ]:
eigvecs

In [ ]:
np.savetxt('Data/EigenAnalysis/PR_vals_quadTdep.csv',eigvals,delimiter=',')
np.savetxt('Data/EigenAnalysis/PR_vecs_quadTdep.csv',eigvecs,delimiter=',')

In [ ]:
FIM = np.linalg.inv(cov) 

#A_opt
FIM_trace = np.trace(FIM)
cov_trace = np.trace(cov)

#D_opt
FIM_det = np.linalg.det(FIM)

#E_opt
eigvals_FIM, eigvecs_FIM = np.linalg.eig(FIM)
eigvals_FIM_min = eigvals_FIM.min()
eigvals_FIM_max = eigvals_FIM.max()

np.savetxt('Data/EigenAnalysis/PR_FIMtrdet_quadTdep.csv',[cov_trace,FIM_trace,FIM_det,eigvals_FIM_min,eigvals_FIM_max])
np.savetxt('Data/EigenAnalysis/PR_FIMvals_quadTdep.csv',eigvals_FIM,delimiter=',')
np.savetxt('Data/EigenAnalysis/PR_FIMvecs_quadTdep.csv',eigvecs_FIM,delimiter=',')

## AIC Value

See: https://doi.org/10.1080/21642583.2018.1496042

AIC(k) = Nln(MSE(k))+2k

N: number of data points

MSE(k): mean squared error of the model with k parameters

k: number of parameters

MSE(k) = SSE(k)/N

SSE: sum of squared error of the model with k parameters

In [ ]:
#number of datapoints
N = len(data_subset)
#MSE(k)
MSE = obj_value*1E-2/N
#number of parameters
k = 6

In [ ]:
AIC = N*np.log(MSE) + 2*k
AIC

In [ ]:
AIC_data = pd.DataFrame(data=[N,MSE,k,obj_value,AIC])
AIC_data = AIC_data.T
AIC_data.columns = ['N','MSE','k','Objective, Pa^2','AIC']
AIC_data.to_csv('Data/AIC/PR_AIC_quadTdep.csv')